In [4]:
# Aggregating power orbit data from operational outputs

import os
import pandas as pd
import re

# Config
input_folder = "_all_modes"
output_csv = "aggregated_power_orbit_table_all_modes.csv"

# Files
files = [f for f in os.listdir(input_folder) if f.endswith("_output.txt")]

# Ordered metrics for final table
ordered_metrics = [
    "ECLIPSE ORBIT PERCENTAGE [%]",
    "SUNLIGHT DURATION [s]",
    "SUNLIGHT ORBIT PERCENTAGE [%]",
    "AVERAGE POWER PER SUNLIGHT [W]",
    "AVERAGE POWER PER ORBIT [W]",
    "AVERAGE POWER PER SUNLIGHT [W],PERCENTAGE OF ORBITS [%]",
    "AVERAGE POWER PER ORBIT [W],PERCENTAGE OF ORBITS [%]"
]

data = {}

for file in files:
    file_path = os.path.join(input_folder, file)
    with open(file_path, "r") as f:
        lines = [line.strip() for line in f if line.strip()]

    values = {}
    i = 0
    while i < len(lines):
        line = lines[i]
        if line in ordered_metrics[:5]:  # metrics with average/min/max
            # Find average value
            j = i + 1
            while j < len(lines) and "AVERAGE" not in lines[j]:
                j += 1
            if j < len(lines):
                avg_match = re.search(r"AVERAGE\s*=\s*([-+]?[0-9]*\.?[0-9]+)", lines[j])
                if avg_match:
                    values[line] = round(float(avg_match.group(1)), 1)
            i = j + 1
        elif line in ordered_metrics[5:]:  # table metrics
            # Get the highest percentage (≈99%)
            j = i + 1
            max_percentage_val = None
            while j < len(lines) and "," in lines[j]:
                val, perc = lines[j].split(",")
                if float(perc) >= 99:
                    max_percentage_val = round(float(val), 1)
                    break
                j += 1
            if max_percentage_val is not None:
                values[line] = max_percentage_val
            i = j + 1
        else:
            i += 1

    data[file] = values

# Create DataFrame with files as columns and metrics as rows
df = pd.DataFrame(data)
df = df.reindex(ordered_metrics)  # enforce row order

# Save CSV
df.to_csv(output_csv)
print(f"Aggregated table saved to {output_csv}")

Aggregated table saved to aggregated_power_orbit_table_all_modes.csv


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import re

# --- Load CSV ---
file_path = "aggregated_power_orbit_table_all_modes.csv"  # update path if needed
df = pd.read_csv(file_path)

# --- Extract the "AVERAGE POWER PER ORBIT [W]" row ---
power_row = df.iloc[4, 1:]  # skip the first column

# --- Parse columns into Altitude, LTAN, Mode, Power ---
data = []
pattern = re.compile(
    r"orbit_(\d+)_ltan_(\d+)_output_(.+?)_output\.txt"
)
for col, val in power_row.items():
    m = pattern.match(col)
    if m:
        alt = int(m.group(1))
        ltan = int(m.group(2))
        mode = m.group(3)  # e.g., SUN_GS, VELOCITY_NADIR
        data.append({
            "Altitude": alt,
            "LTAN": ltan,
            "Mode": mode,
            "Power": float(val)
        })

df_plot = pd.DataFrame(data)

# --- Filter to a single altitude if desired ---
altitude_filter = 500
df_plot = df_plot[df_plot["Altitude"] == altitude_filter]

# --- Sort LTANs and Modes for consistent order ---
ltan_order = sorted(df_plot["LTAN"].unique())
mode_order = sorted(df_plot["Mode"].unique())

# ---------- 1. Grouped by LTAN ----------
plt.figure(figsize=(10, 6))
bar_width = 0.12
for i, mode in enumerate(mode_order):
    subset = df_plot[df_plot["Mode"] == mode]
    subset = subset.set_index("LTAN").reindex(ltan_order)
    x_positions = [x + i * bar_width for x in range(len(ltan_order))]
    plt.bar(x_positions, subset["Power"], width=bar_width, label=mode)

plt.xticks([x + bar_width*(len(mode_order)-1)/2 for x in range(len(ltan_order))],
           [f"LTAN {l}" for l in ltan_order])
plt.xlabel("LTAN")
plt.ylabel("Average Power per Orbit [W]")
plt.title(f"Average Power per Orbit by LTAN (Altitude {altitude_filter} km)")
plt.legend(title="Operational Mode", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.savefig("power_by_ltan.png", dpi=300)
plt.close()

# ---------- 2. Grouped by Operational Mode ----------
plt.figure(figsize=(10, 6))
bar_width = 0.12
for i, ltan in enumerate(ltan_order):
    subset = df_plot[df_plot["LTAN"] == ltan]
    subset = subset.set_index("Mode").reindex(mode_order)
    x_positions = [x + i * bar_width for x in range(len(mode_order))]
    plt.bar(x_positions, subset["Power"], width=bar_width, label=f"LTAN {ltan}")

plt.xticks([x + bar_width*(len(ltan_order)-1)/2 for x in range(len(mode_order))],
           mode_order, rotation=45, ha="right")
plt.xlabel("Operational Mode")
plt.ylabel("Average Power per Orbit [W]")
plt.title(f"Average Power per Orbit by Operational Mode (Altitude {altitude_filter} km)")
plt.legend(title="LTAN", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.savefig("power_by_mode.png", dpi=300)
plt.close()

print("Charts saved: power_by_ltan.png and power_by_mode.png")


Charts saved: power_by_ltan.png and power_by_mode.png


In [6]:
# save as make_pastel_by_altitude.py
import pandas as pd
import matplotlib.pyplot as plt
import re
from math import isfinite

FILE = "aggregated_power_orbit_table_all_modes.csv"

# --- Load ---
df = pd.read_csv(FILE)
power_row = df.iloc[4, 1:]  # "AVERAGE POWER PER ORBIT [W]" row, skip label col

# --- Parse columns -> Altitude, LTAN, Mode, Power ---
rec = []
pat = re.compile(r"orbit_(\d+)_ltan_(\d+)_output_(.+?)_output\.txt")
for col, val in power_row.items():
    m = pat.match(col)
    if not m:
        continue
    try:
        p = float(val)
        if not isfinite(p):
            continue
    except Exception:
        continue
    rec.append({
        "Altitude": int(m.group(1)),
        "LTAN": int(m.group(2)),
        "Mode": m.group(3),
        "Power": float(val),
    })
dfp = pd.DataFrame(rec)

# --- Pastel palettes (stable) ---
PASTEL_MODES = [
    "#aec7e8","#ffbb78","#98df8a","#ff9896","#c5b0d5",
    "#c49c94","#f7b6d2","#c7c7c7","#dbdb8d","#9edae5"
]
PASTEL_LTAN = ["#aec7e8","#ffbb78","#98df8a","#ff9896","#c5b0d5","#c49c94"]

def save_grouped_by_ltan(alt, filename):
    dfa = dfp[dfp["Altitude"] == alt].copy()
    if dfa.empty:
        print(f"No data for {alt} km")
        return
    ltan_order = sorted(dfa["LTAN"].unique())
    mode_order = sorted(dfa["Mode"].unique())
    mode_colors = {m: PASTEL_MODES[i % len(PASTEL_MODES)] for i, m in enumerate(mode_order)}

    bar_w = 0.10
    x = range(len(ltan_order))
    plt.figure(figsize=(11, 6))
    for i, mode in enumerate(mode_order):
        sub = dfa[dfa["Mode"] == mode].set_index("LTAN").reindex(ltan_order)
        xpos = [xx + i*bar_w for xx in x]
        plt.bar(xpos, sub["Power"], width=bar_w, label=mode,
                color=mode_colors[mode], edgecolor="black", linewidth=0.5)

    plt.xticks([xx + bar_w*(len(mode_order)-1)/2 for xx in x], [f"LTAN {lt}" for lt in ltan_order])
    plt.xlabel("LTAN")
    plt.ylabel("Average Power per Orbit [W]")
    plt.title(f"Average Power per Orbit by LTAN (Altitude {alt} km, Pastel)")
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.legend(title="Operational Mode", bbox_to_anchor=(1.02, 1), loc="upper left")
    plt.tight_layout()
    plt.savefig(filename, dpi=300)
    plt.close()
    print(f"Saved {filename}")

def save_grouped_by_mode(alt, filename):
    dfa = dfp[dfp["Altitude"] == alt].copy()
    if dfa.empty:
        print(f"No data for {alt} km")
        return
    ltan_order = sorted(dfa["LTAN"].unique())
    mode_order = sorted(dfa["Mode"].unique())
    ltan_colors = {lt: PASTEL_LTAN[i % len(PASTEL_LTAN)] for i, lt in enumerate(ltan_order)}

    bar_w = 0.10
    x = range(len(mode_order))
    plt.figure(figsize=(11, 6))
    for i, lt in enumerate(ltan_order):
        sub = dfa[dfa["LTAN"] == lt].set_index("Mode").reindex(mode_order)
        xpos = [xx + i*bar_w for xx in x]
        plt.bar(xpos, sub["Power"], width=bar_w, label=f"LTAN {lt}",
                color=ltan_colors[lt], edgecolor="black", linewidth=0.5)

    plt.xticks([xx + bar_w*(len(ltan_order)-1)/2 for xx in x], mode_order, rotation=30, ha="right")
    plt.xlabel("Operational Mode")
    plt.ylabel("Average Power per Orbit [W]")
    plt.title(f"Average Power per Orbit by Operational Mode (Altitude {alt} km, Pastel)")
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.legend(title="LTAN", bbox_to_anchor=(1.02, 1), loc="upper left")
    plt.tight_layout()
    plt.savefig(filename, dpi=300)
    plt.close()
    print(f"Saved {filename}")

# --- Generate four figures ---
save_grouped_by_ltan(500, "power_by_ltan_500_pastel.png")
save_grouped_by_mode(500, "power_by_mode_500_pastel.png")
save_grouped_by_ltan(600, "power_by_ltan_600_pastel.png")
save_grouped_by_mode(600, "power_by_mode_600_pastel.png")


Saved power_by_ltan_500_pastel.png
Saved power_by_mode_500_pastel.png
Saved power_by_ltan_600_pastel.png
Saved power_by_mode_600_pastel.png


In [7]:
# save as make_altitude_pairs.py
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from math import isfinite

FILE = "aggregated_power_orbit_table_all_modes.csv"

# --- Load ---
df = pd.read_csv(FILE)
power_row = df.iloc[4, 1:]  # "AVERAGE POWER PER ORBIT [W]"

# --- Parse columns ---
rec = []
pat = re.compile(r"orbit_(\d+)_ltan_(\d+)_output_(.+?)_output\.txt")
for col, val in power_row.items():
    m = pat.match(col)
    if not m:
        continue
    try:
        p = float(val)
        if not isfinite(p):
            continue
    except Exception:
        continue
    rec.append({
        "Altitude": int(m.group(1)),
        "LTAN": int(m.group(2)),
        "Mode": m.group(3),
        "Power": float(val),
    })
dfp = pd.DataFrame(rec)

# --- Altitude style (consistent across both figures) ---
ALT_PALETTE = {500: "#aec7e8", 600: "#ffbb78"}  # pastel blue/orange
ALT_HATCH   = {500: "//", 600: "\\\\"}
EDGE = "black"; LW = 0.5

def save_grouped_by_ltan_with_alt_pairs(filename):
    dfa = dfp.copy()
    ltan_order = sorted(dfa["LTAN"].unique())
    mode_order = sorted(dfa["Mode"].unique())
    alt_order  = [500, 600]

    bar_w = 0.08
    pair_gap = 0.015
    mode_gap = 0.05
    group_gap = 0.20

    bars = []
    x_cursor = 0.0

    for lt in ltan_order:
        for mode in mode_order:
            for i, alt in enumerate(alt_order):
                x = x_cursor + i * (bar_w + pair_gap)
                v = dfa[(dfa["LTAN"] == lt) & (dfa["Mode"] == mode) & (dfa["Altitude"] == alt)]["Power"].values
                y = float(v[0]) if v.size else 0.0
                bars.append((x, y, ALT_PALETTE[alt], ALT_HATCH[alt]))
            x_cursor += 2*(bar_w + pair_gap) + mode_gap
        x_cursor += group_gap

    # Plot
    plt.figure(figsize=(14, 6))
    for x, h, c, hch in bars:
        plt.bar(x, h, width=bar_w, color=c, edgecolor=EDGE, linewidth=LW, hatch=hch)

    # X ticks per LTAN group
    group_centers = []
    cursor = 0.0
    per_mode_span = 2*(bar_w + pair_gap) + mode_gap
    group_span = len(mode_order)*per_mode_span - mode_gap
    for lt in ltan_order:
        group_centers.append(cursor + group_span/2.0)
        cursor += group_span + group_gap

    plt.xticks(group_centers, [f"LTAN {lt}" for lt in ltan_order])
    plt.xlabel("LTAN (modes grouped within each)")
    plt.ylabel("Average Power per Orbit [W]")
    plt.title("Average Power per Orbit — Altitude Pairs Within Each Mode (Grouped by LTAN)")
    plt.grid(axis="y", linestyle="--", alpha=0.7)

    handles = [plt.Rectangle((0,0),1,1, color=ALT_PALETTE[a], ec=EDGE, lw=LW, hatch=ALT_HATCH[a]) for a in alt_order]
    plt.legend(handles, [f"{a} km" for a in alt_order], title="Altitude", loc="best")

    plt.tight_layout()
    plt.savefig(filename, dpi=300)
    plt.close()
    print(f"Saved {filename}")

def save_grouped_by_mode_with_alt_pairs(filename):
    dfa = dfp.copy()
    mode_order = sorted(dfa["Mode"].unique())
    ltan_order = sorted(dfa["LTAN"].unique())
    alt_order  = [500, 600]

    bar_w = 0.08
    pair_gap = 0.015
    ltan_gap = 0.05
    group_gap = 0.20

    bars = []
    x_cursor = 0.0

    for mode in mode_order:
        for lt in ltan_order:
            for i, alt in enumerate(alt_order):
                x = x_cursor + i * (bar_w + pair_gap)
                v = dfa[(dfa["Mode"] == mode) & (dfa["LTAN"] == lt) & (dfa["Altitude"] == alt)]["Power"].values
                y = float(v[0]) if v.size else 0.0
                bars.append((x, y, ALT_PALETTE[alt], ALT_HATCH[alt]))
            x_cursor += 2*(bar_w + pair_gap) + ltan_gap
        x_cursor += group_gap

    plt.figure(figsize=(14, 6))
    for x, h, c, hch in bars:
        plt.bar(x, h, width=bar_w, color=c, edgecolor=EDGE, linewidth=LW, hatch=hch)

    # X ticks per mode group
    group_centers = []
    cursor = 0.0
    per_ltan_span = 2*(bar_w + pair_gap) + ltan_gap
    group_span = len(ltan_order)*per_ltan_span - ltan_gap
    for mode in mode_order:
        group_centers.append(cursor + group_span/2.0)
        cursor += group_span + group_gap

    plt.xticks(group_centers, mode_order, rotation=25, ha="right")
    plt.xlabel("Operational Mode (LTAN subgroups inside each; altitude pairs per subgroup)")
    plt.ylabel("Average Power per Orbit [W]")
    plt.title("Average Power per Orbit — Altitude Pairs Within Each LTAN (Grouped by Mode)")
    plt.grid(axis="y", linestyle="--", alpha=0.7)

    handles = [plt.Rectangle((0,0),1,1, color=ALT_PALETTE[a], ec=EDGE, lw=LW, hatch=ALT_HATCH[a]) for a in alt_order]
    plt.legend(handles, [f"{a} km" for a in alt_order], title="Altitude", loc="best")

    plt.tight_layout()
    plt.savefig(filename, dpi=300)
    plt.close()
    print(f"Saved {filename}")

# --- Build both comparison figures ---
save_grouped_by_ltan_with_alt_pairs("power_by_ltan_alt_pairs.png")
save_grouped_by_mode_with_alt_pairs("power_by_mode_alt_pairs.png")


Saved power_by_ltan_alt_pairs.png
Saved power_by_mode_alt_pairs.png


In [8]:
import os
import re
import pandas as pd

# Path to folder containing OAP analysis txt files
folder = "_OAP_analysis"

# Regex to capture parameter name and AVERAGE value
param_re = re.compile(r"^(.*?)\s*\nMINIMUM\s*=\s*[\d\.\-Ee+]+\s*\nAVERAGE\s*=\s*([\d\.\-Ee+]+)", re.MULTILINE)

# Store results
rows = []
all_params = set()

for fname in os.listdir(folder):
    if fname.lower().endswith(".txt"):
        fpath = os.path.join(folder, fname)
        with open(fpath, "r", encoding="utf-8") as f:
            content = f.read()

        # Extract parameter name and average
        matches = param_re.findall(content)
        row = {"File": fname}
        for param, avg in matches:
            param_clean = param.strip()
            try:
                avg_val = float(avg)
            except:
                avg_val = None
            row[param_clean] = avg_val
            all_params.add(param_clean)

        rows.append(row)

# Create DataFrame with all possible parameters as columns
df = pd.DataFrame(rows)
# Ensure all parameter columns are present even if missing in some files
df = df[["File"] + sorted(all_params)]

# Save to CSV
output_csv = "OAP_analysis_overview.csv"
df.to_csv(output_csv, index=False)

print(f"Saved overview to {output_csv}")


Saved overview to OAP_analysis_overview.csv


In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import re

# === Load the CSV overview ===
csv_path = "OAP_analysis_overview.csv"  # path to your CSV file
df = pd.read_csv(csv_path)

# Column containing the values to plot
param_col = "AVERAGE POWER PER ORBIT [W]"

# Files to plot (in order)
files_plot = [
    "orbit_500_ltan_10_output_fault_1_VELOCITY_NADIR_output.txt",
    "orbit_500_ltan_10_output_fault_2_VELOCITY_NADIR_output.txt",
    "orbit_500_ltan_10_output_plus10_VELOCITY_NADIR_output.txt",
]

# Helper to make nice labels
pat = re.compile(r"orbit_(\d+)_ltan_(\d+)_output_(.+?)_output\.txt")
def friendly_label(fname: str) -> str:
    m = pat.match(fname)
    if not m:
        return fname
    alt, ltan, middle = m.groups()
    return f"{alt} km LTAN {ltan} — {middle.replace('_', ' ')}"

# Filter and order the data
sub = df[df["File"].isin(files_plot)].copy()
sub["order"] = sub["File"].apply(lambda f: files_plot.index(f))
sub.sort_values("order", inplace=True)

# Labels and values for plotting
labels = [friendly_label(f) for f in sub["File"]]
values = sub[param_col].astype(float).tolist()

# === Plot ===
plt.figure(figsize=(8, 5))
plt.bar(range(len(values)), values, color="#aec7e8", edgecolor="black")
plt.xticks(range(len(labels)), labels, rotation=30, ha="right")
plt.ylabel(param_col)
plt.title("Average Power per Orbit — LTAN 10 (Faults vs plus10, Velocity-Nadir)")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()

# Save to PNG
out_path = "plot1_ltan10_faults_vs_plus10_only.png"
plt.savefig(out_path, dpi=300)
plt.close()

print(f"Saved plot to {out_path}")


Saved plot to plot1_ltan10_faults_vs_plus10_only.png


In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import re

# === Load the CSV overview ===
csv_path = "OAP_analysis_overview.csv"  # path to your CSV file
df = pd.read_csv(csv_path)

# Column containing the values to plot
param_col = "AVERAGE POWER PER ORBIT [W]"

# Files to plot (in order)
files_plot = [
    "orbit_500_ltan_9_output_SUN_GS_output.txt",
    "orbit_500_ltan_9_output_SUN_NADIR_output.txt",
    "orbit_500_ltan_9_output_SUN_VELOCITY_output.txt",
    "orbit_500_ltan_9_output_VELOCITY_GS_output.txt",
    "orbit_500_ltan_9_output_VELOCITY_NADIR_output.txt",
    "orbit_500_ltan_9_output_VELOCITY_SUN_output.txt",
    "orbit_600_ltan_12_output_SUN_GS_output.txt",
    "orbit_600_ltan_12_output_SUN_NADIR_output.txt",
    "orbit_600_ltan_12_output_SUN_VELOCITY_output.txt",
    "orbit_600_ltan_12_output_VELOCITY_GS_output.txt",
    "orbit_600_ltan_12_output_VELOCITY_NADIR_output.txt",
    "orbit_600_ltan_12_output_VELOCITY_SUN_output.txt",
    "orbit_600_ltan_15_output_SUN_GS_output.txt",
    "orbit_600_ltan_15_output_SUN_NADIR_output.txt",
    "orbit_600_ltan_15_output_SUN_VELOCITY_output.txt",
    "orbit_600_ltan_15_output_VELOCITY_GS_output.txt",
    "orbit_600_ltan_15_output_VELOCITY_NADIR_output.txt",
    "orbit_600_ltan_15_output_VELOCITY_SUN_output.txt",
    "orbit_600_ltan_9_output_SUN_GS_output.txt",
    "orbit_600_ltan_9_output_SUN_NADIR_output.txt",
    "orbit_600_ltan_9_output_SUN_VELOCITY_output.txt",
    "orbit_600_ltan_9_output_VELOCITY_GS_output.txt",
    "orbit_600_ltan_9_output_VELOCITY_NADIR_output.txt",
    "orbit_600_ltan_9_output_VELOCITY_SUN_output.txt",
    "orbit_500_ltan_12_output_SUN_GS_output.txt",
    "orbit_500_ltan_12_output_SUN_NADIR_output.txt",
    "orbit_500_ltan_12_output_SUN_VELOCITY_output.txt",
    "orbit_500_ltan_12_output_VELOCITY_GS_output.txt",
    "orbit_500_ltan_12_output_VELOCITY_NADIR_output.txt",
    "orbit_500_ltan_12_output_VELOCITY_SUN_output.txt",
    "orbit_500_ltan_15_output_SUN_GS_output.txt",
    "orbit_500_ltan_15_output_SUN_NADIR_output.txt",
    "orbit_500_ltan_15_output_SUN_VELOCITY_output.txt",
    "orbit_500_ltan_15_output_VELOCITY_GS_output.txt",
    "orbit_500_ltan_15_output_VELOCITY_NADIR_output.txt",
    "orbit_500_ltan_15_output_VELOCITY_SUN_output.txt",
]

# Helper to make nice labels
pat = re.compile(r"orbit_(\d+)_ltan_(\d+)_output_(.+?)_output\.txt")
def friendly_label(fname: str) -> str:
    m = pat.match(fname)
    if not m:
        return fname
    alt, ltan, middle = m.groups()
    return f"{alt} km LTAN {ltan} — {middle.replace('_', ' ')}"

# Filter and order the data
sub = df[df["File"].isin(files_plot)].copy()
sub["order"] = sub["File"].apply(lambda f: files_plot.index(f))
sub.sort_values("order", inplace=True)

# Labels and values for plotting
labels = [friendly_label(f) for f in sub["File"]]
values = sub[param_col].astype(float).tolist()

# === Plot ===
plt.figure(figsize=(14, 6))
plt.bar(range(len(values)), values, color="#aec7e8", edgecolor="black")
plt.xticks(range(len(labels)), labels, rotation=70, ha="right")
plt.ylabel(param_col)
plt.title("Average Power per Orbit — Multiple LTANs, Altitudes, and Modes")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()

# Save to PNG
out_path = "plot_modes_multiple_ltan_altitudes.png"
plt.savefig(out_path, dpi=300)
plt.close()

print(f"Saved plot to {out_path}")


Saved plot to plot_modes_multiple_ltan_altitudes.png


In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re

# === Load CSV ===
csv_path = "OAP_analysis_overview.csv"
df = pd.read_csv(csv_path)

param_col = "AVERAGE POWER PER ORBIT [W]"

# Filenames to include (your list)
files_plot = [
    "orbit_500_ltan_9_output_SUN_GS_output.txt",
    "orbit_500_ltan_9_output_SUN_NADIR_output.txt",
    "orbit_500_ltan_9_output_SUN_VELOCITY_output.txt",
    "orbit_500_ltan_9_output_VELOCITY_GS_output.txt",
    "orbit_500_ltan_9_output_VELOCITY_NADIR_output.txt",
    "orbit_500_ltan_9_output_VELOCITY_SUN_output.txt",
    "orbit_600_ltan_12_output_SUN_GS_output.txt",
    "orbit_600_ltan_12_output_SUN_NADIR_output.txt",
    "orbit_600_ltan_12_output_SUN_VELOCITY_output.txt",
    "orbit_600_ltan_12_output_VELOCITY_GS_output.txt",
    "orbit_600_ltan_12_output_VELOCITY_NADIR_output.txt",
    "orbit_600_ltan_12_output_VELOCITY_SUN_output.txt",
    "orbit_600_ltan_15_output_SUN_GS_output.txt",
    "orbit_600_ltan_15_output_SUN_NADIR_output.txt",
    "orbit_600_ltan_15_output_SUN_VELOCITY_output.txt",
    "orbit_600_ltan_15_output_VELOCITY_GS_output.txt",
    "orbit_600_ltan_15_output_VELOCITY_NADIR_output.txt",
    "orbit_600_ltan_15_output_VELOCITY_SUN_output.txt",
    "orbit_600_ltan_9_output_SUN_GS_output.txt",
    "orbit_600_ltan_9_output_SUN_NADIR_output.txt",
    "orbit_600_ltan_9_output_SUN_VELOCITY_output.txt",
    "orbit_600_ltan_9_output_VELOCITY_GS_output.txt",
    "orbit_600_ltan_9_output_VELOCITY_NADIR_output.txt",
    "orbit_600_ltan_9_output_VELOCITY_SUN_output.txt",
    "orbit_500_ltan_12_output_SUN_GS_output.txt",
    "orbit_500_ltan_12_output_SUN_NADIR_output.txt",
    "orbit_500_ltan_12_output_SUN_VELOCITY_output.txt",
    "orbit_500_ltan_12_output_VELOCITY_GS_output.txt",
    "orbit_500_ltan_12_output_VELOCITY_NADIR_output.txt",
    "orbit_500_ltan_12_output_VELOCITY_SUN_output.txt",
    "orbit_500_ltan_15_output_SUN_GS_output.txt",
    "orbit_500_ltan_15_output_SUN_NADIR_output.txt",
    "orbit_500_ltan_15_output_SUN_VELOCITY_output.txt",
    "orbit_500_ltan_15_output_VELOCITY_GS_output.txt",
    "orbit_500_ltan_15_output_VELOCITY_NADIR_output.txt",
    "orbit_500_ltan_15_output_VELOCITY_SUN_output.txt",
]

# Keep only those files
sub = df[df["File"].isin(files_plot)].copy()

# Parse filename -> Altitude, LTAN, Mode
pat = re.compile(r"orbit_(\d+)_ltan_(\d+)_output_(.+?)_output\.txt")
def parse_parts(fname):
    m = pat.match(fname)
    if not m:
        return None, None, None
    alt, ltan, mode = m.groups()
    return int(alt), int(ltan), mode

sub[["Altitude","LTAN","Mode"]] = sub["File"].apply(lambda f: pd.Series(parse_parts(f)))

# Orders
ltan_order = sorted(sub["LTAN"].unique())
mode_order = ["SUN_GS","SUN_NADIR","SUN_VELOCITY","VELOCITY_GS","VELOCITY_NADIR","VELOCITY_SUN"]
alt_order  = [500, 600]

# Shortened mode labels for the minor ticks
mode_short = {
    "SUN_GS":"SGS", "SUN_NADIR":"SND", "SUN_VELOCITY":"SV",
    "VELOCITY_GS":"VGS", "VELOCITY_NADIR":"VNA", "VELOCITY_SUN":"VS"
}

# Pastel colours by MODE
mode_colors = {
    "SUN_GS":          "#aec7e8",
    "SUN_NADIR":       "#ffbb78",
    "SUN_VELOCITY":    "#98df8a",
    "VELOCITY_GS":     "#ff9896",
    "VELOCITY_NADIR":  "#c5b0d5",
    "VELOCITY_SUN":    "#c49c94",
}
# Hatching by altitude
alt_hatch = {500: "", 600: "//"}

# Geometry
bar_w   = 0.08
pair_gap = 0.01          # between 500/600 within a mode
mode_gap = 0.05          # between modes within LTAN
group_gap = 0.20         # between LTAN groups

# Build bars + tick positions
bars = []                # (x, y, facecolor, hatch)
group_centers = []       # for LTAN labels (major ticks)
minor_centers = []       # for mode labels (minor ticks)
minor_labels  = []

x_cursor = 0.0
for lt in ltan_order:
    start_of_group = x_cursor
    for mode in mode_order:
        # Bars for 500/600 at this mode
        for i, alt in enumerate(alt_order):
            x = x_cursor + i * (bar_w + pair_gap)
            match = sub[(sub["LTAN"] == lt) & (sub["Mode"] == mode) & (sub["Altitude"] == alt)]
            y = float(match.iloc[0][param_col]) if not match.empty else np.nan
            bars.append((x, y, mode_colors.get(mode, "#cccccc"), alt_hatch[alt]))
        # Center of this 500/600 pair for minor label
        mode_center = x_cursor + (bar_w + pair_gap) * 0.5
        minor_centers.append(mode_center)
        minor_labels.append(mode_short.get(mode, mode))
        x_cursor += 2*(bar_w + pair_gap) + mode_gap
    # span of this LTAN (subtract trailing mode_gap)
    group_span = 6 * (2*(bar_w + pair_gap) + mode_gap) - mode_gap
    group_centers.append(start_of_group + group_span / 2.0)
    x_cursor += group_gap

# === Plot with extra margins so labels/legend don’t collide ===
fig, ax = plt.subplots(figsize=(18, 7))

# Bars
for x, h, color, hatch in bars:
    ax.bar(x, h, width=bar_w, color=color, edgecolor="black", linewidth=0.5, hatch=hatch)

# Major ticks: LTANs (push down a bit so they don't overlay minor labels)
ax.set_xticks(group_centers)
ax.set_xticklabels([f"LTAN {lt}" for lt in ltan_order])
ax.tick_params(axis="x", which="major", pad=28, labelsize=11)

# Minor ticks: short mode labels under each LTAN group
ax.set_xticks(minor_centers, minor=True)
ax.set_xticklabels(minor_labels, minor=True)
ax.tick_params(axis="x", which="minor", pad=8, labelsize=9, length=0)

# Labels, title, grid
ax.set_xlabel("LTAN (each group shows modes; 500/600 bars per mode)")
ax.set_ylabel(param_col)
ax.set_title("Average Power per Orbit — Grouped by LTAN (Mode-coloured; 600 km hatched)")
ax.grid(axis="y", linestyle="--", alpha=0.7)

# Legends placed outside on the right
mode_handles = [plt.Rectangle((0,0),1,1, facecolor=mode_colors[m], edgecolor="black", linewidth=0.5) for m in mode_order]
alt_handles  = [plt.Rectangle((0,0),1,1, facecolor="#dddddd", edgecolor="black", linewidth=0.5, hatch=alt_hatch[a]) for a in alt_order]

first_legend = ax.legend(mode_handles, mode_order, title="Operational Mode",
                         loc="upper left", bbox_to_anchor=(1.02, 1))
ax.add_artist(first_legend)
ax.legend(alt_handles, [f"{a} km" for a in alt_order], title="Altitude",
          loc="lower left", bbox_to_anchor=(1.02, 0))

# Make room on the right for legends and on the bottom for the two-tier ticks
plt.subplots_adjust(right=0.80, bottom=0.22)

# Save
out_path = "plot_modes_grouped_by_ltan_mode_coloured_with_mode_labels_v2.png"
plt.savefig(out_path, dpi=300)
plt.close()

print(f"Saved {out_path}")


Saved plot_modes_grouped_by_ltan_mode_coloured_with_mode_labels_v2.png


In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re

# === Load CSV ===
csv_path = "OAP_analysis_overview.csv"
df = pd.read_csv(csv_path)

param_col = "AVERAGE POWER PER ORBIT [W]"

# Files to include (your list)
files_plot = [
    "orbit_500_ltan_10_output_plus10_SUN_GS_output.txt",
    "orbit_500_ltan_10_output_plus10_SUN_NADIR_output.txt",
    "orbit_500_ltan_10_output_plus10_SUN_VELOCITY_output.txt",
    "orbit_500_ltan_10_output_plus10_VELOCITY_GS_output.txt",
    "orbit_500_ltan_10_output_plus10_VELOCITY_NADIR_output.txt",
    "orbit_500_ltan_10_output_plus10_VELOCITY_SUN_output.txt",
]

sub = df[df["File"].isin(files_plot)].copy()
if sub.empty:
    raise RuntimeError("None of the specified files were found in the CSV. Check filenames/paths.")

# ---- Parse filename -> Altitude, LTAN, Cant, Mode ----
# 'middle' can be "plus10_SUN_GS" or "fault_1_VELOCITY_NADIR" or just "SUN_GS"
pat = re.compile(r"orbit_(\d+)_ltan_(\d+)_output_(.+?)_output\.txt")
mode_tail = re.compile(r"(SUN_(?:GS|NADIR|VELOCITY)|VELOCITY_(?:GS|NADIR|SUN))$")

def parse_parts(fname):
    m = pat.match(fname)
    if not m:
        return None, None, None, None
    alt, ltan, middle = m.groups()
    mm = mode_tail.search(middle)
    if mm:
        mode = mm.group(1)
        cant = middle[: mm.start()].strip("_")  # prefix before the mode
        cant = cant or None
    else:
        # Fallback: treat whole middle as mode if no match (rare)
        mode, cant = middle, None
    return int(alt), int(ltan), cant, mode

sub[["Altitude","LTAN","Cant","Mode"]] = sub["File"].apply(lambda f: pd.Series(parse_parts(f)))

# ---- Orders (auto-detect, but keep preferred mode order when present) ----
preferred_mode_order = ["SUN_GS","SUN_NADIR","SUN_VELOCITY","VELOCITY_GS","VELOCITY_NADIR","VELOCITY_SUN"]
mode_order = [m for m in preferred_mode_order if m in set(sub["Mode"])]
if not mode_order:
    mode_order = sorted(sub["Mode"].unique())

ltan_order = sorted(sub["LTAN"].unique())         # likely [10]
alt_order  = sorted(sub["Altitude"].unique())     # likely [500]

# ---- Colours by MODE (pastel) + short labels ----
mode_colors = {
    "SUN_GS":          "#aec7e8",
    "SUN_NADIR":       "#ffbb78",
    "SUN_VELOCITY":    "#98df8a",
    "VELOCITY_GS":     "#ff9896",
    "VELOCITY_NADIR":  "#c5b0d5",
    "VELOCITY_SUN":    "#c49c94",
}
mode_short = {
    "SUN_GS":"SGS", "SUN_NADIR":"SND", "SUN_VELOCITY":"SV",
    "VELOCITY_GS":"VGS", "VELOCITY_NADIR":"VNA", "VELOCITY_SUN":"VS"
}
alt_hatch = {alt: ("//" if alt == 600 else "") for alt in alt_order}

# ---- Geometry ----
bar_w   = 0.10
pair_gap = 0.015
mode_gap = 0.08
group_gap = 0.25

# ---- Build bars & ticks ----
bars = []                # (x, y, color, hatch)
group_centers = []
minor_centers = []
minor_labels  = []

x_cursor = 0.0
for lt in ltan_order:
    group_start = x_cursor
    for mode in mode_order:
        # bars for each altitude actually present
        for i, alt in enumerate(alt_order):
            x = x_cursor + i * (bar_w + pair_gap)
            match = sub[(sub["LTAN"] == lt) & (sub["Mode"] == mode) & (sub["Altitude"] == alt)]
            if not match.empty and pd.notna(match.iloc[0][param_col]):
                y = float(match.iloc[0][param_col])
                bars.append((x, y, mode_colors.get(mode, "#cccccc"), alt_hatch[alt]))
        # minor tick per mode subgroup
        width_used = len(alt_order) * (bar_w + pair_gap) - pair_gap if alt_order else 0
        mode_center = x_cursor + max(width_used, bar_w) / 2.0
        minor_centers.append(mode_center)
        minor_labels.append(mode_short.get(mode, mode))
        x_cursor += len(alt_order) * (bar_w + pair_gap) + mode_gap
    span = len(mode_order)*(len(alt_order)*(bar_w+pair_gap)+mode_gap) - mode_gap
    group_centers.append(group_start + span/2.0)
    x_cursor += group_gap

# ---- Plot ----
fig, ax = plt.subplots(figsize=(12, 5))

for x, h, color, hatch in bars:
    ax.bar(x, h, width=bar_w, color=color, edgecolor="black", linewidth=0.5, hatch=hatch)

# Major LTAN ticks (spaced from minor)
ax.set_xticks(group_centers)
ax.set_xticklabels([f"LTAN {lt}" for lt in ltan_order])
ax.tick_params(axis="x", which="major", pad=26, labelsize=11)

# Minor mode ticks
ax.set_xticks(minor_centers, minor=True)
ax.set_xticklabels(minor_labels, minor=True)
ax.tick_params(axis="x", which="minor", pad=6, labelsize=9, length=0)

ax.set_xlabel("LTAN (grouped). Within each group, mode-coloured bars" + (", 500/600 pairs" if len(alt_order)>1 else ""))
ax.set_ylabel(param_col)
ax.set_title("Average Power per Orbit — LTAN grouped, mode-coloured (plus10 case)")
ax.grid(axis="y", linestyle="--", alpha=0.7)

# Legend (mode colours). Altitude legend only if multiple altitudes.
mode_handles = [plt.Rectangle((0,0),1,1, facecolor=mode_colors[m], edgecolor="black", linewidth=0.5)
                for m in mode_order]
first_legend = ax.legend(mode_handles, mode_order, title="Operational Mode",
                         loc="upper left", bbox_to_anchor=(1.01, 1))
ax.add_artist(first_legend)

if len(alt_order) > 1:
    alt_handles = [plt.Rectangle((0,0),1,1, facecolor="#dddddd", edgecolor="black", linewidth=0.5,
                                 hatch=("//" if a==600 else "")) for a in alt_order]
    ax.legend(alt_handles, [f"{a} km" for a in alt_order], title="Altitude",
              loc="lower left", bbox_to_anchor=(1.01, 0))

plt.subplots_adjust(right=0.8, bottom=0.22)

# Save
out_path = "ltan10_plus10_modes.png"
plt.savefig(out_path, dpi=300)
plt.close()

print(f"Saved {out_path}")


Saved ltan10_plus10_modes.png
